# Test for game with observabilities

In [22]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [23]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner

import source.util as util
import numpy as np
import pandas as pd
from importlib import *
import source.util as util
from copy import deepcopy, copy

In [24]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
reload(environment)
reload(util)

<module 'source.util' from 'C:\\Users\\Mattia\\Desktop\\Mattia\\Git\\onlineSG\\source\\util.py'>

In [25]:
time_horizon = 1000

In [26]:
T = 5 #np.random.randint(2,10)
targets = util.gen_targets(T, low=1, high=10)
distributions = []
for i in range(20):
    distributions.append(tuple(util.gen_distr(T)))
values = tuple((v, v) for v in targets)
print("Values:", values)
observabilities = {0: 0.5, 1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5}
print("Observabilities:", observabilities)
g = game.Game(values, time_horizon)
g_o = game.GameWithObservabilities(values, time_horizon)
g_o.set_observabilities(observabilities)

#generating attacker, defender and profiles for game without observabilities
attacker = [attackers.StackelbergAttacker(g, 1, 1)]
profiles = []
profiles.append(attackers.StackelbergAttacker(g, 1, 1))
for d in distributions:
    profiles.append(attackers.StochasticAttacker(g, 1, 1, *d))
profiles.append(attackers.StackelbergAttacker(g, 1))
defender = [base_defenders.StackelbergDefender(g, 0, 1)]
g.set_players(defender, attacker, profiles)

#generating attacker, defender and profiles for game with observabilities
attacker_o = [attackers.ObservingStackelbergAttacker(g_o, 1, 1)]
profiles_o = []
profiles_o.append(attackers.ObservingStackelbergAttacker(g_o, 1, 1))
for d in distributions:
    profiles_o.append(attackers.StochasticAttacker(g_o, 1, 1, *d))
#adding standard Stackelberg attacker to compare its behavior wrt Observing one
profiles_o.append(attackers.StackelbergAttacker(g_o, 1))
defender_o = [base_defenders.StackelbergDefender(g_o, 0, 1)]
g_o.set_players(defender_o, attacker_o, profiles_o)

Values: ((2, 2), (5, 5), (8, 8), (4, 4), (1, 1))
Observabilities: {0: 0.5, 1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5}


In [27]:
e = runner.Experiment(g)
e_o = runner.Experiment(g_o)

In [28]:
e.run_interaction()
e_o.run_interaction_with_observations()

[{0: [0.0, 0.30434782608695654, 0.5652173913043479, 0.13043478260869557, 0.0]}]
[{0: [0.0, 0.30434782608695654, 0.5652173913043479, 0.13043478260869557, 0.0]}]


In [29]:
print(e.game.strategy_history[-1])
print(e.game.history[-1])
print(e.environment.last_exp_loss)
print(e.environment.last_opt_loss)

{0: [0.0, 0.30434782608695654, 0.5652173913043479, 0.13043478260869557, 0.0], 1: [0, 0, 0, 1, 0]}
{1: [3], 0: [2]}
3.4782608695652177
3.478260869565216


In [30]:
print(e_o.game.strategy_history[-1])
print(e_o.game.history[-1])
print(e_o.game.observation_history[-1])
print(e_o.environment.last_exp_loss)
print(e_o.environment.last_opt_loss)

{0: [0.0, 0.30434782608695654, 0.5652173913043479, 0.13043478260869557, 0.0], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [1]}
[(1, 0)]
3.478260869565217
3.4782608695652177


In [10]:
isinstance(e_o.game, game.GameWithObservabilities)

True